In [1]:
import torch
import os
import pickle
from tqdm import tqdm
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [3]:
TRUTHFUL_ACOUSTIC_PATH = '../../data/Acoustic/Truthful/'
DECEPTIVE_ACOUSTIC_PATH = '../../data/Acoustic/Deceptive/'
TRUTHFUL_ACOUSTIC_FILES = os.listdir(TRUTHFUL_ACOUSTIC_PATH)
DECEPTIVE_ACOUSTIC_FILES = os.listdir(DECEPTIVE_ACOUSTIC_PATH)

In [4]:
DECEPTIVE_LEXICAL_PATH = '../../data/Transcription/Deceptive/'
DECEPTIVE_LEXICAL_FILES = os.listdir(DECEPTIVE_LEXICAL_PATH)
TRUTHFUL_LEXICAL_PATH = '../../data/Transcription/Truthful/'
TRUTHFUL_LEXICAL_FILES = os.listdir(TRUTHFUL_LEXICAL_PATH)

In [53]:
class LexicalAndAcousticClassifier(torch.nn.Module):
    def __init__(self, embedding_size):
        super(LexicalAndAcousticClassifier, self).__init__()
        self.linear1 = torch.nn.Linear(embedding_size, 16)
        self.linear2 = torch.nn.Linear(16, 2)
        torch.nn.init.kaiming_uniform_(self.linear1.weight)
        torch.nn.init.xavier_uniform_(self.linear2.weight)
    
    def forward(self, x):
        x = self.linear1(torch.nn.functional.relu(x))
        return self.linear2(torch.nn.functional.relu(x))

In [33]:
class LexicalAndAcousticDataset(Dataset):
    def __init__(self, lexical_data, acoustic_data, lexical_file, acoustic_file, pca_l, sc_l, pca_a, sc_a, test):
        self.lexical_embeddings = []
        self.labels = []
        with open(lexical_file, 'rb') as f:
            self.embedding_map = pickle.load(f)
        for file in lexical_data:
            embedding = self.embedding_map[file.split('.')[0]]
            if type(embedding) == np.ndarray:
                self.lexical_embeddings.append(embedding)
            else:
                self.lexical_embeddings.append(embedding.numpy())
        self.lexical_embeddings = np.array(self.lexical_embeddings)
        with open(acoustic_file, 'rb') as f:
            embeddings_map = pickle.load(f)
        self.acoustic_embeddings = np.array([embeddings_map[file] for file in acoustic_data])
        if not test:
            self.lexical_embeddings = sc_l.fit_transform(self.lexical_embeddings)
            self.lexical_embeddings = pca_l.fit_transform(self.lexical_embeddings)
            self.acoustic_embeddings = sc_a.fit_transform(self.acoustic_embeddings)
            self.acoustic_embeddings = pca_a.fit_transform(self.acoustic_embeddings)
        else:
            self.lexical_embeddings = sc_l.transform(self.lexical_embeddings)
            self.lexical_embeddings = pca_l.transform(self.lexical_embeddings)
            self.acoustic_embeddings = sc_a.transform(self.acoustic_embeddings)
            self.acoustic_embeddings = pca_a.transform(self.acoustic_embeddings)
        self.lexical_embeddings = torch.from_numpy(self.lexical_embeddings).float()
        self.acoustic_embeddings = torch.from_numpy(self.acoustic_embeddings).float()
        self.labels = [0 if file.split('_')[1] == 'truth' else 1 for file in acoustic_data]
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        return torch.concat((self.lexical_embeddings[index], self.acoustic_embeddings[index])), self.labels[index]

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:
def train(model, train_loader, criterion, optimizer, num_epochs):
    for _ in range(num_epochs):
        model.train()
        for x, y in train_loader:
            x = x.to(device)
            y = y.to(device)
            out = model(x)
            loss = criterion(out, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [36]:
def eval(model, val_loader):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for x, y in val_loader:
            x = x.to(device)
            y = y.to(device)
            out = model(x)
            _, predicted = torch.max(out.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
        return correct / total

In [37]:
def kfold(lexical_file, acoustic_file, train_batch_size, embedding_size):
    truthful_acoustic_data = list(TRUTHFUL_ACOUSTIC_FILES)
    deceptive_acoustic_data = list(DECEPTIVE_ACOUSTIC_FILES)
    truthful_lexical_data = list(TRUTHFUL_LEXICAL_FILES)
    deceptive_lexical_data = list(DECEPTIVE_LEXICAL_FILES)
    for i in range(10):
        val_acoustic_data = truthful_acoustic_data[i*6:(i+1)*6] + deceptive_acoustic_data[i*6:(i+1)*6]
        train_acoustic_data = truthful_acoustic_data[:i*6] + truthful_acoustic_data[(i+1)*6:] + deceptive_acoustic_data[:i*6] + deceptive_acoustic_data[(i+1)*6:]
        val_lexical_data = truthful_lexical_data[i*6:(i+1)*6] + deceptive_lexical_data[i*6:(i+1)*6]
        train_lexical_data = truthful_lexical_data[:i*6] + truthful_lexical_data[(i+1)*6:] + deceptive_lexical_data[:i*6] + deceptive_lexical_data[(i+1)*6:]
        sc_a = StandardScaler()
        pca_a = PCA(n_components=embedding_size // 2)
        sc_l = StandardScaler()
        pca_l = PCA(n_components=embedding_size // 2)
        train_dataset = LexicalAndAcousticDataset(train_lexical_data, train_acoustic_data, lexical_file, acoustic_file, pca_l, sc_l, pca_a, sc_a, False)
        val_dataset = LexicalAndAcousticDataset(val_lexical_data, val_acoustic_data, lexical_file, acoustic_file, pca_l, sc_l, pca_a, sc_a, True)
        train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=12)
        yield train_loader, val_loader

In [38]:
def run_one_fold(train_loader, val_loader, num_epochs, embedding_size):
    model = LexicalAndAcousticClassifier(embedding_size)
    model.to(device)
    learning_rate = 1e-3
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    train(model, train_loader, criterion, optimizer, num_epochs)
    return eval(model, val_loader)

In [39]:
def get_model_accuracy(lexical_file, acoustic_file, train_batch_size, num_epochs, embedding_size = 96):
    accuracies = []
    for train_loader, val_loader in kfold(lexical_file, acoustic_file, train_batch_size, embedding_size):
        accuracies.append(run_one_fold(train_loader, val_loader, num_epochs, embedding_size))
    return sum(accuracies) / len(accuracies)

In [66]:
# Model accuracy for bert-base-uncased and hubert-large-ls960-ft 
get_model_accuracy('../../embeddings/transcript_features_1.pkl', '../../embeddings/acoustic_features_1.pkl', 4, 20, 64)

0.55

In [67]:
# Model accuracy for bert-base-cased-finetuned-emotion and hubert-large-ls960-ft 
get_model_accuracy('../../embeddings/transcript_features_2.pkl', '../../embeddings/acoustic_features_1.pkl', 4, 20, 64)

0.6249999999999999

In [68]:
# Model accuracy for transformersbook/distilbert-base-uncased-finetuned-emotion and hubert-large-ls960-ft 
get_model_accuracy('../../embeddings/transcript_features_3.pkl', '../../embeddings/acoustic_features_1.pkl', 4, 20, 64)

0.6083333333333333

In [69]:
# Model accuracy for mpNet and hubert-large-ls960-ft 
get_model_accuracy('../../embeddings/transcript_features_6.pkl', '../../embeddings/acoustic_features_1.pkl', 4, 20, 64)

0.5416666666666666